# Scripts

In this notebook we present all possible scripts.

In [1]:
import pandas as pd
from mammos_mumag.materials import Materials
from mammos_mumag.parameters import Parameters
from mammos_mumag.simulation import Simulation

/home/mlang/repos/mammos/mammos-mumag/.pixi/envs/docs/lib/python3.11/site-packages/mammos_entity/base.py:15: RuntimeWarning: Failed to load ontology from the internet.
  from mammos_entity.onto import HAVE_INTERNET, mammos_ontology
/home/mlang/repos/mammos/mammos-mumag/.pixi/envs/docs/lib/python3.11/site-packages/mammos_entity/base.py:135: RuntimeWarning: Failed to load ontology from the interent. Hence, no check for unit or ontology_label will be performed!
  warnings.warn(


## Mesh, materials, simulation parameters

### Mesh

We load the mesh of a cube enclosed with a sphere. This sphere is embedded in a further spherical shell.

This mesh was generated by Salome in the `unv` mesh, and successively converted in the `fly` format. As an input to the `Simulation` class we give the attribute `mesh_filepath`.

### Material parameters

We define the material parameters, defined by 3 domains: the internal geometry (a cube), the sphere and the infinity shell.

### Simulation parameters

We finally define all the simulation parameters we will use.
> Note: most of these parameters are the "default" parameters and in fact we do not need to define them.

In [2]:
sim = Simulation(
    mesh_filepath="data/cube.fly",
    materials=Materials(
        domains=[
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 4.9e06,
                "K2": 0.0,
                "Ms": 1.61,
                "A": 8.0e-11,
            },
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 0.0,
                "K2": 0.0,
                "Ms": 0.0,
                "A": 0.0,
            },
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 0.0,
                "K2": 0.0,
                "Ms": 0.0,
                "A": 0.0,
            },
        ],
    ),
    parameters=Parameters(
        size=1.0e-9,
        scale=0,
        m_vect=[0, 0, 1],
        hstart=1,
        hfinal=-1,
        hstep=-0.2,
        h_vect=[0.01745, 0, 0.99984],
        mstep=0.4,
        mfinal=-1.2,
        tol_fun=1e-10,
        tol_hmag_factor=1,
        precond_iter=10,
    ),
)
sim

Simulation(mesh_filepath=PosixPath('data/cube.fly'), materials=Materials(domains=[MaterialDomain(theta=0.0, phi=0.0, K1=UniaxialAnisotropyConstant(value=4900000.0, unit=J / m3), K2=UniaxialAnisotropyConstant(value=0.0, unit=J / m3), Ms=SpontaneousMagnetization(value=1.61, unit=A / m), A=ExchangeStiffnessConstant(value=8e-11, unit=J / m)), MaterialDomain(theta=0.0, phi=0.0, K1=UniaxialAnisotropyConstant(value=0.0, unit=J / m3), K2=UniaxialAnisotropyConstant(value=0.0, unit=J / m3), Ms=SpontaneousMagnetization(value=0.0, unit=A / m), A=ExchangeStiffnessConstant(value=0.0, unit=J / m)), MaterialDomain(theta=0.0, phi=0.0, K1=UniaxialAnisotropyConstant(value=0.0, unit=J / m3), K2=UniaxialAnisotropyConstant(value=0.0, unit=J / m3), Ms=SpontaneousMagnetization(value=0.0, unit=A / m), A=ExchangeStiffnessConstant(value=0.0, unit=J / m))]), parameters=Parameters(size=1e-09, scale=0.0, state='', m_vect=[0.0, 0.0, 1.0], hmag_on=1, hstart=1.0, hfinal=-1.0, hstep=-0.2, h_vect=[0.01745, 0.0, 0.99984]

Note that all of this could also have been defined using file paths:
```python
sim = Simulation(
    mesh_filepath="data/cube.fly",
    materials_filepath="data/cube.krn",
    parameters_filepath="data/cube.p2",
)
```

## Available scripts
For all scripts we can specify the optional variables `outdir` and `name`. While the first identifies the output directory where the input and output files will be store (and where the script is executed), the `name` variable defines the output file names.

### Save the mesh and the materials

To create the `vtk` file for the visualisation of the material properties we can use the script `materials`.

In [3]:
sim.run_materials(outdir="out/materials", name="cube")

This create discretized representations of the material scalar functions and fields in the mesh and stores the file `out/materials/cube_mat.vtu`.
This object contains information about the mesh and the scalar and vectorial fields defined on it.

### Compute the magnetostatic field

To create the `vtk` file for visualisation of the magnetic scalar potential and the magnetic field we use the script `hmag`.
With linear basis function for the magnetic scalar potential $u$, the magnetostatic field $h = - \nabla u$ is defined at the finite elements. By smoothing the field can be transfered to the nodes of the finite element mesh. This is `h_at_nodes`.

In [4]:
sim.run_hmag(outdir="out/hmag", name="cube")

The output (configuration of the demagnetisation field) `cube_hmag.vtu` will be stored in the output directory `out/hmag`.

The scripts creates two file: the magnetostatic field, as seen above, will be stored in `cube_hmag.vtu`.
At the same time the software also gives the magnetostatic energy density computed with finite elements and compares it with the analytic solution:
- `from field`:
  \begin{equation}
  E_{\mathsf{field}} := \frac{1}{2} \int_\Omega \frac{\mathbf{h} \cdot J_s \mathbf{m}}{V} \ \mathrm{d}x,
  \end{equation}
  where $\Omega$ is the domain, $\mathbf{h}$ is the demagnetization field, $J_s$ is the spontaneeous polarization, $\mathbf{m}$ is the magnetization field, and $V$ is the volume of the domain.
- `from_gradient`:
  \begin{equation}
  \frac{1}{2} \sum_i \mathbf{m}_i \cdot \mathbf{g}_i,
  \end{equation}
  where $\mathbf{m}_i$ and $\mathbf{g}_i$ are the unit vector of the magnetization and the gradient of the energy normalized by the volume of the energy with respect to $\mathbf{m}_i$ at the nodes of the finite element mesh.
- `analytic`: $J_s^2 / (6 \mu_0)$

This information will be saved in `cube.csv`:

In [5]:
pd.read_csv("out/hmag/cube.csv", skiprows=1)

,name,value,explanation
0,E_field,5.101095e-07,Energy density evaluated from field (J/m^3).
1,E_gradient,5.101095e-07,Energy density evaluated from gradient (J/m^3).
2,E_analytic,5.428882e-07,Energy density evaluated analytically (J/m^3).


### Exchange and anisotropy energy

To test the computation of the exchange and anisotropy energy density we can use the script `exani`.

This gives the exchange energy density of a vortex in the $xy$-plane and the anistropy energy density in the uniformly magnetized state.
Here we have placed the anistropy direction paralle to to the $z$-axis. The anisotropy energy density is calculated as $-K (\mathbf{m} \cdot \mathbf{k})^2$  where $\mathbf{m}$ is the unit vector of magnetization and $\mathbf{k}$ is the anisotropy direction. $K$ is the magnetocrystalline anisotropy constant

In [6]:
sim.run_exani(outdir="out/exani", name="cube")

The accuracy of this script is then analyzed for two different magnetization, a vortex and a uniform vector.

In [7]:
pd.read_csv("out/exani/cube_vortex.csv", skiprows=1)

,name,value,explanation
0,E_gradient,983624.055448,Energy evaluated from gradient (J/m^3).
1,E_analytic,986960.440109,Energy evaluated analytically (J/m^3).


In [8]:
pd.read_csv("out/exani/cube_uniform.csv", skiprows=1)

,name,value,explanation
0,E_gradient,-4.900000e+06,Energy evaluated from gradient (J/m^3).
1,E_analytic,-4.900000e+06,Energy evaluated analytically (J/m^3).


### Zeeman energy

The script `external` calculates the Zeeman energy density for an external field of $\mu_0 H_{\mathsf{ext}} = 1.2 \ T$ by finite elements and analytically.

In [9]:
sim.run_external(outdir="out/external", name="cube")

The generated energy densities are found in the generated file `cube.csv`:

In [10]:
pd.read_csv("out/external/cube.csv", skiprows=1)

,name,value,explanation
0,E_gradient,-1.931706,Energy evaluated from gradient (J/m^3).
1,E_analytic,-1.931706,Energy evaluated analytically (J/m^3).


### jax implementation

The above tools checked the energy calculation with the finite element backend.
From the finite element backend system matrices are generated for micromagnetic simulations.
The script `mapping` is used to test the energy calculations with matrices.

The `mumag` software uses sparse matrix methods from `jax`.

In [11]:
sim.run_mapping(outdir="out/mapping", name="cube")

Information about the calculation are stored in different files. Among them, `cube_energy.csv` stores the total energy density for the uniformly magnetized state:

In [12]:
pd.read_csv("out/mapping/cube_energy.csv", skiprows=1)

,name,value,explanation
0,E_jax,-4899998.0,Energy evaluated with jax backend (J/m^3).
1,E_analytic,-4899998.0,Energy evaluated analytically (J/m^3).


The file `cube_stats.txt`, on the other hand, shows information about memory and runtime.

In [13]:
with open("out/mapping/cube_stats.txt") as file:
    print(file.read())

MAP FINITE ELEMENT BACKEND (esys-escript) TO JAX.
Memory before escript2jax: 178.0390625 MB.
Memory after  escript2jax: 462.89453125 MB.
Memory after garbage collection: 462.89453125 MB.
Timing and statistics.
elapsed time: 0.3762063980102539
function_calls: 1


### Storing sparse matrices

The sparse matrices used for computation can be stored and reused for simulations with the same finite element mesh. To store the matrices use the script `store`.

In [14]:
sim.run_store(outdir="out/store", name="cube")

### Demagnetization curve - Hysteresis Loop

To compute the demagnetization curve we use the script `loop`.

In [15]:
sim.run_loop(outdir="out/loop", name="cube")

This creates the file `cube.dat` which gives the demagnetization curve. The columns of the file are:
- `vtk number`: the number of the `vtk` file that corresponds to the field and magnetic polarisation values in the line.
- `mu0 Hext`: the value of $\mu_0 H_{\mathsf{ext}}$ ($T$) where $\mu_0$ is the permability of vacuum and $H_{\mathsf{ext}}$ is the external value of the external field.
- `polarisation`: the componenent of magnetic polarisation ($T$) parallel to the direction of the external field.
- `energy density`: the energy density ($J/m^3$) of the current state.

In [16]:
pd.read_csv(
    "out/loop/cube.dat", delimiter=" ", names=["idx", "mu0_Hext", "polarisation", "E"]
)

,idx,mu0_Hext,polarisation,E
0,1,1.000000e+00,0.000002,-4.900002e+06
1,1,8.000000e-01,0.000002,-4.900001e+06
2,1,6.000000e-01,0.000002,-4.900001e+06
3,1,4.000000e-01,0.000002,-4.900001e+06
4,1,2.000000e-01,0.000002,-4.900000e+06
5,1,5.551115e-17,0.000002,-4.900000e+06
6,1,-2.000000e-01,0.000002,-4.900000e+06
7,1,-4.000000e-01,0.000002,-4.899999e+06
8,1,-6.000000e-01,0.000002,-4.899999e+06
9,1,-8.000000e-01,0.000002,-4.899999e+06


Further execution statistics are found in file `cube_stats.txt`

In [17]:
with open("out/loop/cube_stats.txt") as file:
    print(file.read())

Memory before escript2jax: 177.96484375 MB.
Memory after  escript2jax: 428.95703125 MB.
Memory after garbage collection: 428.95703125 MB.


And several configurations are stored as `vtu` files in `out/loop/`.